## Hierarchical Agent Teams

In our previous example ([Agent Supervisor](./agent_supervisor.ipynb)), we introduced the concept of a single supervisor node to route work between different worker nodes.

But what if the job for a single worker becomes too complex? What if the number of workers becomes too large?

For some applications, the system may be more effective if work is distributed _hierarchically_.

You can do this by composing different subgraphs and creating a top-level supervisor, along with mid-level supervisors.

To do this, let's build a simple research assistant! The graph will look something like the following:

![diagram](./img/hierarchical-diagram.png)

This notebook is inspired by the paper [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155), by Wu, et. al. In the rest of this notebook, you will:

1. Define the agents' tools to access the web and write files
2. Define some utilities to help create the graph and agents
3. Create and define each team (web research + doc writing)
4. Compose everything together.

But before all of that, some setup:

In [5]:
#  %%capture --no-stderr
%pip install -U langgraph langchain langchain_openai langchain_experimental matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from getpass import getpass
import os
import uuid


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith.
# This will help you visualize and debug the control flow
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

## Create Tools

Each team will be composed of one or more agents each with one or more tools. Below, define all the tools to be used by your different teams.

We'll start with the research team.

**Research team tools**

The research team can use a search engine and url scraper to find information on the web. Feel free to add additional functionality below to boost the team performance!

In [2]:
from typing import Annotated, List, Tuple, Union

import matplotlib.pyplot as plt
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langsmith import trace

tavily_tool = TavilySearchResults(max_results=5)


@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

**Document writing team tools**

Next up, we will give some tools for the doc writing team to use.
We define some bare-bones file-access tools below.

Note that this gives the agents access to your file-system, which can be unsafe. We also haven't optimized the tool descriptions for performance.

In [3]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional

from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict

_TEMP_DIRECTORY = TemporaryDirectory()
LOCAL_WORKING_DIRECTORY = Path(_TEMP_DIRECTORY.name)
print(LOCAL_WORKING_DIRECTORY)
WORKING_DIRECTORY = Path("./work_dir")


@tool
def create_outline(
    points: Annotated[List[str], "List of main points or sections."],
    file_name: Annotated[str, "File path to save the outline."],
) -> Annotated[str, "Path of the saved outline file."]:
    """Create and save an outline."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        for i, point in enumerate(points):
            file.write(f"{i + 1}. {point}\n")
    return f"Outline saved to {file_name}"


@tool
def read_document(
    file_name: Annotated[str, "File path to save the document."],
    start: Annotated[Optional[int], "The start line. Default is 0"] = None,
    end: Annotated[Optional[int], "The end line. Default is None"] = None,
) -> str:
    """Read the specified document."""
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    if start is not None:
        start = 0
    return "\n".join(lines[start:end])


@tool
def write_document(
    content: Annotated[str, "Text content (can be code) to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document (can be code)."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"


@tool
def edit_document(
    file_name: Annotated[str, "Path of the document (can be code) to be edited."],
    inserts: Annotated[
        Dict[int, str],
        "Dictionary where key is the line number (1-indexed) and value is the text to be inserted at that line.",
    ],
) -> Annotated[str, "Path of the edited document file."]:
    """Edit a document by inserting text at specific line numbers."""

    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()

    sorted_inserts = sorted(inserts.items())

    for line_number, text in sorted_inserts:
        if 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, text + "\n")
        else:
            return f"Error: Line number {line_number} is out of range."

    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.writelines(lines)

    return f"Document edited and saved to {file_name}"


# Warning: This executes code locally, which can be unsafe when not sandboxed

repl = PythonREPL()


@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your tools."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\n```python\n{code}\n```\nStdout: {result}"

/var/folders/d0/f5cdt48n3hbc76z8tk2rpg6w0000gn/T/tmpuy9f9gjf


## Helper Utilities

We are going to create a few utility functions to make it more concise when we want to:

1. Create a worker agent.
2. Create a supervisor for the sub-graph.

These will simplify the graph compositional code at the end for us so it's easier to see what's going on.

In [4]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph


def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification. Take your best guess given the available information."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

## Define Agent Teams

Now we can get to define our hierachical teams. "Choose your player!"

### Research Team

The research team will have a search agent and a web scraping "research_agent" as the two worker nodes. Let's create those, as well as the team supervisor.

In [5]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools


# Research team graph state
class ResearchTeamState(TypedDict):
    # A message is added after each team member finishes
    messages: Annotated[List[BaseMessage], operator.add]
    # The team members are tracked so they are aware of
    # the others' skill-sets
    team_members: List[str]
    # Used to route work. The supervisor calls a function
    # that will update this every time it makes a decision
    next: str


llm = ChatOpenAI(model="gpt-4-1106-preview")

search_agent = create_agent(
    llm,
    [tavily_tool],
    "You are a research assistant who can search for up-to-date info, library documentation, code solutions, or other references using the tavily search engine.",
)
search_node = functools.partial(agent_node, agent=search_agent, name="Search")

research_agent = create_agent(
    llm,
    [scrape_webpages],
    "You are a research assistant who can scrape specified urls for more detailed information using the scrape_webpages function.",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Web Scraper")

supervisor_agent = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Search, Web Scraper. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Search", "Web Scraper"],
)

Now that we've created the necessary components, defining their interactions is easy. Add the nodes to the team graph, and define the edges, which determine the transition criteria.

In [6]:
research_graph = StateGraph(ResearchTeamState)
research_graph.add_node("Search", search_node)
research_graph.add_node("Web Scraper", research_node)
research_graph.add_node("supervisor", supervisor_agent)

# Define the control flow
research_graph.add_edge("Search", "supervisor")
research_graph.add_edge("Web Scraper", "supervisor")
research_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"Search": "Search", "Web Scraper": "Web Scraper", "FINISH": END},
)


research_graph.set_entry_point("supervisor")
chain = research_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


research_chain = enter_chain | chain

We can give this team work directly. Try it out below.

In [13]:
for s in research_chain.stream(
    """
   Creating a specification for integrating Swaks (Swiss Army Knife for SMTP) with a Large Language Model (LLM) for testing and analyzing SMTP server functionality involves automating the process of sending test emails, capturing the output of these tests, and then processing this data for insights. Swaks is a versatile command-line tool that's ideal for this task, thanks to its ability to simulate various email sending scenarios and capture detailed SMTP session data.

### Specification for Analyzing SMTP Functionality with Swaks and an LLM

#### Objective:
Develop an automated process to use Swaks for generating SMTP test emails, capturing the transaction details, and analyzing these details with an LLM to identify potential issues, optimizations, or confirm functionality of the SMTP setup.

#### Requirements:

1. **Swaks Installation**:
   - Ensure Swaks is installed on the system where the tests will be run. If not installed, Swaks can typically be installed via package managers on Unix-like systems.

2. **SMTP Test Configuration**:
   - Define SMTP server details, including the host, port, and any authentication details if necessary.
   - Specify the sender and recipient email addresses for the test emails.

3. **Automated Swaks Execution**:
   - Use Swaks to send test emails, capturing the detailed output of the SMTP transaction.
   - Optionally, use various Swaks options to simulate different email sending scenarios.

4. **Data Structuring for LLM Analysis**:
   - Process the Swaks output to extract and structure key data points from the SMTP transaction in a format suitable for LLM analysis, such as JSON.

5. **LLM Integration for Analysis**:
   - Send the structured SMTP transaction data to the LLM for analysis.
   - Interpret the LLM's output to extract actionable insights regarding SMTP server performance, configuration issues, or security concerns.

6. **Actionable Insights Implementation**:
   - Based on the LLM's analysis, implement or suggest specific actions to address identified issues or optimizations.

#### Implementation:

```bash
#!/bin/bash

# Define SMTP server details
SMTP_SERVER="smtp.example.com"
SMTP_PORT=25
SENDER="sender@example.com"
RECIPIENT="recipient@example.com"

# Define the path for the structured output file
STRUCTURED_OUTPUT="/path/to/swaks-output.json"

# Use Swaks to send a test email and capture the output
swaks_output=$(swaks --server $SMTP_SERVER --port $SMTP_PORT --to $RECIPIENT --from $SENDER --hide-all)

# Process the Swaks output to extract relevant data and structure it for LLM analysis
# This is a simplified example. You may need to adjust the parsing to fit your needs
echo "{ \"smtp_server\": \"$SMTP_SERVER\", \"smtp_port\": $SMTP_PORT, \"sender\": \"$SENDER\", \"recipient\": \"$RECIPIENT\", \"transaction_details\": \"$swaks_output\" }" > $STRUCTURED_OUTPUT

echo "Structured SMTP transaction data saved to $STRUCTURED_OUTPUT"

# Placeholder for sending structured data to the LLM and retrieving analysis
# You will need to implement this part based on how you interact with the LLM
# Example placeholder:
# python3 analyze_with_llm.py "$STRUCTURED_OUTPUT"

echo "LLM analysis complete. Review recommendations and take action as necessary."
```

Before running this script:
- Ensure Swaks is installed on your system.
- Replace placeholders (`smtp.example.com`, `sender@example.com`, `recipient@example.com`, `/path/to/swaks-output.json`) with actual values relevant to your test.
- The processing of Swaks output in this example is very basic. Depending on your needs and the format of the LLM analysis, you may need to extract and structure the data more precisely.
- Implement the logic for sending structured data to the LLM and processing its output according to the specifics of your LLM's API or interface.

This specification aims to facilitate automated testing and analysis of SMTP functionality using Swaks and an LLM, providing a basis for identifying and addressing issues within your SMTP server setup. """
    , {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'FINISH'}}
---


### Document Writing Team

Create the document writing team below using a similar approach. This time, we will give each agent access to different file-writing tools.

Note that we are giving file-system access to our agent here, which is not safe in all cases.

In [14]:
import operator
from pathlib import Path


# Document writing team graph state
class DocWritingState(TypedDict):
    # This tracks the team's conversation internally
    messages: Annotated[List[BaseMessage], operator.add]
    # This provides each worker with context on the others' skill sets
    team_members: str
    # This is how the supervisor tells langgraph who to work next
    next: str
    # This tracks the shared directory state
    current_files: str


# This will be run before each worker agent begins work
# It makes it so they are more aware of the current state
# of the working directory.
def prelude(state):
    written_files = []
    if not WORKING_DIRECTORY.exists():
        WORKING_DIRECTORY.mkdir()
    try:
        written_files = [
            f.relative_to(WORKING_DIRECTORY) for f in WORKING_DIRECTORY.rglob("*")
        ]
    except:
        pass
    if not written_files:
        return {**state, "current_files": "No files written."}
    return {
        **state,
        "current_files": "\nBelow are files your team has written to the directory:\n"
        + "\n".join([f" - {f}" for f in written_files]),
    }


llm = ChatOpenAI(model="gpt-4-1106-preview")

code_writer_agent = create_agent(
    llm,
    [write_document, edit_document, read_document],
    "You are an expert writer of many kinds of code. You write Shell Scripts, Python Scripts, Markdown files, Makefiles, and other types of code as needed.\n"
    # The {current_files} value is populated automatically by the graph state
    "Below are files currently in your directory:\n{current_files}",
)
# Injects current directory working state before each call
context_aware_code_writer_agent = prelude | code_writer_agent
code_writing_node = functools.partial(
    agent_node, agent=context_aware_code_writer_agent, name="Expert Programmer"
)

code_planning_agent = create_agent(
    llm,
    [create_outline, read_document],
    "You are an expert senior programmer tasked with writing a project outline and"
    " taking notes from the documentation you have read, and creating a psudeocode / documentation priming to aid the coder in writing each individual file. Please keep track of any code examples, math equations, tables, formulae, or anything else that you might put on a reference on how to develop the requested tool. {current_files}",
)
context_aware_code_planning_agent = prelude | code_planning_agent
code_planning_node = functools.partial(
    agent_node, agent=context_aware_code_planning_agent, name="Tool Architect"
)

code_critic = create_agent(
    llm,
    [read_document, python_repl],
    "You are an LLM tool critic, tasked with reviewing the code other AIs have written and giving reflective feedback on how to improve the code. Do not add functionality beyond what is specified but do ensure it is not incomplete, needlessly complicated, insecure, or over-broad in scope in such a way as to be potentially dangerous. Try to make sure we don't destroy the world with our code. {current_files}",
)
context_aware_code_critic = prelude | code_critic
chart_generating_node = functools.partial(
    agent_node, agent=context_aware_code_critic, name="Code Critic"
)

code_writing_supervisor = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following teammates:  {team_members}. Given the following user request,"
    " respond with the teammate to act next. Each teammate will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Expert Programmer", "Tool Architect", "Code Critic"],
)

In [15]:
import operator
from pathlib import Path


# Document writing team graph state
class DocWritingState(TypedDict):
    # This tracks the team's conversation internally
    messages: Annotated[List[BaseMessage], operator.add]
    # This provides each worker with context on the others' skill sets
    team_members: str
    # This is how the supervisor tells langgraph who to work next
    next: str
    # This tracks the shared directory state
    current_files: str


# This will be run before each worker agent begins work
# It makes it so they are more aware of the current state
# of the working directory.
def prelude(state):
    written_files = []
    if not WORKING_DIRECTORY.exists():
        WORKING_DIRECTORY.mkdir()
    try:
        written_files = [
            f.relative_to(WORKING_DIRECTORY) for f in WORKING_DIRECTORY.rglob("*")
        ]
    except:
        pass
    if not written_files:
        return {**state, "current_files": "No files written."}
    return {
        **state,
        "current_files": "\nBelow are files your team has written to the directory:\n"
        + "\n".join([f" - {f}" for f in written_files]),
    }


llm = ChatOpenAI(model="gpt-4-1106-preview")

code_writer_agent = create_agent(
    llm,
    [write_document, edit_document, read_document],
    "You are an expert writer of many kinds of code. You write Shell Scripts, Python Scripts, Markdown files, Makefiles, and other types of code as needed.\n"
    # The {current_files} value is populated automatically by the graph state
    "Below are files currently in your directory:\n{current_files}",
)
# Injects current directory working state before each call
context_aware_code_writer_agent = prelude | code_writer_agent
code_writing_node = functools.partial(
    agent_node, agent=context_aware_code_writer_agent, name="Expert Programmer"
)

code_planning_agent = create_agent(
    llm,
    [create_outline, read_document],
    "You are an expert senior programmer tasked with writing a project outline and"
    " taking notes from the documentation you have read, and creating a psudeocode / documentation priming to aid the coder in writing each individual file. Please keep track of any code examples, math equations, tables, formulae, or anything else that you might put on a reference on how to develop the requested tool. {current_files}",
)
context_aware_code_planning_agent = prelude | code_planning_agent
code_planning_node = functools.partial(
    agent_node, agent=context_aware_code_planning_agent, name="Tool Architect"
)

code_critic = create_agent(
    llm,
    [read_document, python_repl],
    "You are an LLM tool critic, tasked with reviewing the code other AIs have written and giving reflective feedback on how to improve the code. Do not add functionality beyond what is specified but do ensure it is not incomplete, needlessly complicated, insecure, or over-broad in scope in such a way as to be potentially dangerous. Try to make sure we don't destroy the world with our code. {current_files}",
)
context_aware_code_critic = prelude | code_critic
chart_generating_node = functools.partial(
    agent_node, agent=context_aware_code_critic, name="Code Critic"
)

code_writing_supervisor = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following teammates:  {team_members}. Given the following user request,"
    " respond with the teammate to act next. Each teammate will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Expert Programmer", "Tool Architect", "Code Critic"],
)

In [17]:
# Create the graph here:
# Note that we have unrolled the loop for the sake of this doc
authoring_graph = StateGraph(DocWritingState)

authoring_graph.add_node("Search", search_node)
authoring_graph.add_node("Web Scraper", research_node)
authoring_graph.add_node("search_supervisor", supervisor_agent)

# Define the control flow
authoring_graph.add_edge("Search", "search_supervisor")
authoring_graph.add_edge("Web Scraper", "search_supervisor")


authoring_graph.add_node("Tool Architect", code_planning_node)
authoring_graph.add_node("Expert Programmer", code_writing_node)
authoring_graph.add_node("Code Critic", chart_generating_node)
authoring_graph.add_node("authoring_supervisor", code_writing_supervisor)

# Add the edges that always occur
authoring_graph.add_edge("Tool Architect", "authoring_supervisor")
authoring_graph.add_edge("Expert Programmer", "authoring_supervisor")
authoring_graph.add_edge("Code Critic", "authoring_supervisor")
authoring_graph.add_edge("authoring_supervisor", "search_supervisor")
authoring_graph.add_edge("search_supervisor", "authoring_supervisor")


## after the search supervisor is done, we can pass on the state to the authoring graph
authoring_graph.add_conditional_edges(
    "search_supervisor",
    lambda x: x["next"],
    {
        "Search": "Search", 
        "Web Scraper": "Web Scraper", 
        "FINISH": "authoring_supervisor"
    },
)

# Add the edges where routing applies
authoring_graph.add_conditional_edges(
    "authoring_supervisor",
    lambda x: x["next"],
    {
        "ENTER": "Tool Architect",
        "Tool Architect": "Expert Programmer",
        "Expert Programmer": "Code Critic",
        "Code Critic": "Expert Programmerrammerrammerrammerrammer",
        "FINISH": END,
    },
)

authoring_graph.set_entry_point("search_supervisor")
chain = research_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str, members: List[str]):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ", ".join(members),
    }
    return results


# We re-use the enter/exit functions to wrap the graph
authoring_chain = (
    functools.partial(enter_chain, members=authoring_graph.nodes)
    | authoring_graph.compile()
)

With the objects themselves created, we can form the graph.

In [19]:
for s in authoring_chain.stream(
    """
   Creating a specification for integrating Swaks (Swiss Army Knife for SMTP) with a Large Language Model (LLM) for testing and analyzing SMTP server functionality involves automating the process of sending test emails, capturing the output of these tests, and then processing this data for insights. Swaks is a versatile command-line tool that's ideal for this task, thanks to its ability to simulate various email sending scenarios and capture detailed SMTP session data.

### Specification for Analyzing SMTP Functionality with Swaks and an LLM

#### Objective:
Develop an automated process to use Swaks for generating SMTP test emails, capturing the transaction details, and analyzing these details with an LLM to identify potential issues, optimizations, or confirm functionality of the SMTP setup.

#### Requirements:

1. **Swaks Installation**:
   - Ensure Swaks is installed on the system where the tests will be run. If not installed, Swaks can typically be installed via package managers on Unix-like systems.

2. **SMTP Test Configuration**:
   - Define SMTP server details, including the host, port, and any authentication details if necessary.
   - Specify the sender and recipient email addresses for the test emails.

3. **Automated Swaks Execution**:
   - Use Swaks to send test emails, capturing the detailed output of the SMTP transaction.
   - Optionally, use various Swaks options to simulate different email sending scenarios.

4. **Data Structuring for LLM Analysis**:
   - Process the Swaks output to extract and structure key data points from the SMTP transaction in a format suitable for LLM analysis, such as JSON.

5. **LLM Integration for Analysis**:
   - Send the structured SMTP transaction data to the LLM for analysis.
   - Interpret the LLM's output to extract actionable insights regarding SMTP server performance, configuration issues, or security concerns.

6. **Actionable Insights Implementation**:
   - Based on the LLM's analysis, implement or suggest specific actions to address identified issues or optimizations.

#### Implementation:

```bash
#!/bin/bash

# Define SMTP server details
SMTP_SERVER="smtp.example.com"
SMTP_PORT=25
SENDER="sender@example.com"
RECIPIENT="recipient@example.com"

# Define the path for the structured output file
STRUCTURED_OUTPUT="/path/to/swaks-output.json"

# Use Swaks to send a test email and capture the output
swaks_output=$(swaks --server $SMTP_SERVER --port $SMTP_PORT --to $RECIPIENT --from $SENDER --hide-all)

# Process the Swaks output to extract relevant data and structure it for LLM analysis
# This is a simplified example. You may need to adjust the parsing to fit your needs
echo "{ \"smtp_server\": \"$SMTP_SERVER\", \"smtp_port\": $SMTP_PORT, \"sender\": \"$SENDER\", \"recipient\": \"$RECIPIENT\", \"transaction_details\": \"$swaks_output\" }" > $STRUCTURED_OUTPUT

echo "Structured SMTP transaction data saved to $STRUCTURED_OUTPUT"

# Placeholder for sending structured data to the LLM and retrieving analysis
# You will need to implement this part based on how you interact with the LLM
# Example placeholder:
# python3 analyze_with_llm.py "$STRUCTURED_OUTPUT"

echo "LLM analysis complete. Review recommendations and take action as necessary."
```

Before running this script:
- Ensure Swaks is installed on your system.
- Replace placeholders (`smtp.example.com`, `sender@example.com`, `recipient@example.com`, `/path/to/swaks-output.json`) with actual values relevant to your test.
- The processing of Swaks output in this example is very basic. Depending on your needs and the format of the LLM analysis, you may need to extract and structure the data more precisely.
- Implement the logic for sending structured data to the LLM and processing its output according to the specifics of your LLM's API or interface.

This specification aims to facilitate automated testing and analysis of SMTP functionality using Swaks and an LLM, providing a basis for identifying and addressing issues within your SMTP server setup. 
    """,
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("---")

{'search_supervisor': {'next': 'FINISH'}}
---
{'authoring_supervisor': {'next': 'Tool Architect'}}
---


## Add Layers

In this design, we are enforcing a top-down planning policy. We've created two graphs already, but we have to decide how to route work between the two.

We'll create a _third_ graph to orchestrate the previous two, and add some connectors to define how this top-level state is shared between the different graphs.

In [ ]:
# from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
# from langchain_openai.chat_models import ChatOpenAI


# llm = ChatOpenAI(model="gpt-4-1106-preview")

# supervisor_node = create_team_supervisor(
#     llm,
#     "You are a supervisor tasked with managing a conversation between the"
#     " following teams: {team_members}. Given the following user request,"
#     " respond with the worker to act next. Each worker will perform a"
#     " task and respond with their results and status. When finished,"
#     " respond with FINISH.",
#     ["Research team", "Paper writing team"],
# )

In [ ]:
# Top-level graph state
class State(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next: str


def get_last_message(state: State) -> str:
    return state["messages"][-1].content


def join_graph(response: dict):
    return {"messages": [response["messages"][-1]]}


# Define the graph.
super_graph = StateGraph(State)
# First add the nodes, which will do the work
super_graph.add_node("Research team", get_last_message | research_chain | join_graph)
super_graph.add_node(
    "Paper writing team", get_last_message | authoring_chain | join_graph
)
super_graph.add_node("supervisor", supervisor_node)

# Define the graph connections, which controls how the logic
# propagates through the program
super_graph.add_edge("Research team", "supervisor")
super_graph.add_edge("Paper writing team", "supervisor")
super_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Paper writing team": "Paper writing team",
        "Research team": "Research team",
        "FINISH": END,
    },
)
super_graph.set_entry_point("supervisor")
super_graph = super_graph.compile()

In [ ]:
# for s in super_graph.stream(
#     {
#         "messages": [
#             HumanMessage(
#                 content="Write a brief research report on the North American sturgeon. Include a chart."
#             )
#         ],
#     },
#     {"recursion_limit": 150},
# ):
#     if "__end__" not in s:
#         print(s)
#         print("---")

{'supervisor': {'next': 'Research team'}}
---
{'Research team': {'messages': [HumanMessage(content='# Research Report: North American Sturgeon\n\n## Overview\nSturgeons are ancient fish that have existed for approximately 200 million years. They have distinctive features like a heterocercal caudal fin, bony scutes, and a largely cartilaginous skeleton. They are bottom-feeders, with a protrusible mouth that allows them to suck up their food. Sturgeons are known for their roe, which is processed into caviar - a highly prized and expensive product.\n\n## North American Species\nIn North America, notable species of sturgeon include the Pallid Sturgeon (*Scaphirhynchus albus*), White Sturgeon (*Acipenser transmontanus*), Green Sturgeon (*Acipenser medirostris*), and Shortnose Sturgeon (*Acipenser brevirostrum*). The White Sturgeon is the largest freshwater fish in North America, found from Mexico to Alaska, while the Shortnose Sturgeon is found along the eastern coast from New Brunswick to 

In [ ]:
%pip freeze > ./requirements.txt

Note: you may need to restart the kernel to use updated packages.
